In [1]:
import pandas as pd
import numpy as np
import pymssql

In [2]:
mssql_conn = pymssql.connect(server='epdm', user='sa', password='Pdmepdme1!',database='ePDM_Elcam')

In [3]:
query='select * from dbo.UserRevs'
UserRevs=pd.read_sql(query,mssql_conn)

c:\Users\matan.s\Anaconda3\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [4]:
query2='select * from dbo.Documents'
Documents=pd.read_sql(query2,mssql_conn)
Documents

c:\Users\matan.s\Anaconda3\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,DocumentID,Filename,LockProject,UserID,LockDomain,LockPath,Busy,Flushed,DefValStored,RevGenCounter,...,ExtensionID,LockDate,UserDocRefsModified,Deleted,Shared,LockViewID,Link,DocTypeID,ObjectTypeID,Flags
0,1,No Document,2,1,None,None,False,0,0,0.0,...,1.0,NaT,False,False,0,None,False,1.0,0,0
1,5,pl020120000.slddrw,2,1,,,False,2,1,NaN,...,3.0,NaT,False,False,1,None,False,1.0,1,0
2,6,pl020100000.slddrw,2,1,,,False,2,1,NaN,...,3.0,NaT,False,False,1,None,False,1.0,1,0
3,7,pl020110003.slddrw,2,1,,,False,2,1,NaN,...,3.0,NaT,False,False,1,None,False,1.0,1,0
4,8,sp000768.sldprt,2,1,,,False,2,1,NaN,...,5.0,NaT,False,False,1,None,False,1.0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
472279,547127,582377.slddrw: Bill of Materials1,2,1,,,False,1,1,NaN,...,1.0,NaT,None,False,0,None,False,1.0,3,0
472280,547128,582377.slddrw: Bill of Materials1,2,1,,,False,1,1,NaN,...,1.0,NaT,None,False,0,None,False,1.0,3,0
472281,547129,582377.slddrw: Bill of Materials1,2,1,,,False,1,1,NaN,...,1.0,NaT,None,False,0,None,False,1.0,3,0
472282,547130,582377.slddrw: Bill of Materials1,2,1,,,False,1,1,NaN,...,1.0,NaT,None,False,0,None,False,1.0,3,0


In [21]:
q='select * from dbo.Status'
status=pd.read_sql(q,mssql_conn)
doc_status=pd.merge(Documents,status,left_on='CurrentStatusID',right_on='StatusID')
doc_status
# doc+status

c:\Users\matan.s\Anaconda3\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,DocumentID,Filename,LockProject,UserID,LockDomain,LockPath,Busy,Flushed,DefValStored,RevGenCounter,...,BoxYPos,BoxWidth,BoxHeight,Enabled,Description,RevNumID,BlockRights,WorkflowID,StatusType,RevisionVarID
0,1,No Document,2,1,None,None,False,0,0,0.0,...,0,0,0,False,1,NaN,False,1,3,NaN
1,5,pl020120000.slddrw,2,1,,,False,2,1,NaN,...,1463,194,65,True,,1.0,False,10,3,NaN
2,6,pl020100000.slddrw,2,1,,,False,2,1,NaN,...,1463,194,65,True,,1.0,False,10,3,NaN
3,7,pl020110003.slddrw,2,1,,,False,2,1,NaN,...,1463,194,65,True,,1.0,False,10,3,NaN
4,9,pl020120004.sldprt,2,1,,,False,2,1,NaN,...,1463,194,65,True,,1.0,False,10,3,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
467956,540205,נזילות בליפיד-01-31-123913.pdf,93682,51,MNENG-UDI,C:\ePDM_Elcam\Validations\Val-1830-RX3440 Vs. ...,False,1,1,NaN,...,0,0,0,True,None,NaN,False,1,1,NaN
467957,540217,סאלי 22-01-2023.xlsx,93412,166,QA-SALLY-HP,C:\ePDM_Elcam\Risk Management\Products\STP+Mon...,False,1,1,NaN,...,0,0,0,True,None,NaN,False,1,1,NaN
467958,540371,1.10-1076-Val-Val-1641-New Mold for FDG Handle...,2,1,,,False,2,1,NaN,...,253,210,55,True,,NaN,False,140,3,NaN
467959,540578,Copy of PRJ-0670-Flexi-Q EAI 100 for Epinephri...,91147,561,QA-CHEZI-HP,C:\ePDM_Elcam\Projects\PRJ-0670-Flexi-Q EAI 10...,False,1,1,NaN,...,0,0,0,True,None,NaN,False,1,1,NaN


In [27]:
q="""select Filename  ,s.Name ,ur.Comment 
from Documents d 
full join Status s on s.StatusID =d.CurrentStatusID 
full join UserRevs ur on d.DocumentID =ur.DocumentID 
where Filename like '%.sldprt' or Filename like '%.slddrw%'"""
df=pd.read_sql(q,mssql_conn)
#drop all rows where the comment is None

df

c:\Users\matan.s\Anaconda3\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,Filename,Name,Comment
0,pl020100000.slddrw,Released,0
1,pl020100000.slddrw,Released,A
2,pl020100000.slddrw,Released,B
3,pl020110004.sldprt,Released,B
4,pl020110004.sldprt,Released,A
...,...,...,...
203082,Volumenkörper1-7.sldprt,Under Design,None
203083,Volumenkörper1-9.sldprt,Under Design,None
203084,Solid1.sldprt,Under Design,None
203085,Volumenkörper1-18.sldprt,Under Design,None


In [ ]:
query3="""SELECT DISTINCT ValueCache
FROM VariableValue VV

INNER JOIN

(SELECT DocumentID, ConfigurationID, VariableID, MAX(RevisionNo) AS RevMax

FROM VariableValue
WHERE VariableID=(SELECT VariableID FROM Variable WHERE VariableName= 'PrcdrOrderNum&Subject')
GROUP BY DocumentID,ConfigurationID,VariableID) T

ON VV.DocumentID=T.DocumentID
AND VV.ConfigurationID=T.ConfigurationID
AND VV.VariableID=T.VariableID
AND VV.RevisionNo=T.RevMax
And VV.ValueCache != ''

INNER JOIN

(SELECT D.DocumentID
FROM Projects P,
          DocumentsInProjects P1,
          Documents D,
          Revisions R,
          Status S,
          FileExtension E
WHERE P1.ProjectID=P.ProjectID AND
	  P1.DocumentID=D.DocumentID AND 
	  P.Path LIKE '%' AND
	  D.DocumentID=R.DocumentID AND
D.DocumentID!=55910 AND
	  D.Filename LIKE '%'AND
	  E.ExtensionID=D.ExtensionID AND
	  R.RevNr=D.LatestRevisionNo AND
	  D.CurrentStatusID=S.StatusID AND
	  S.Name NOT IN ('Obsolete') AND
	  D.Deleted='FALSE') Z

ON VV.DocumentID=Z.DocumentID

ORDER BY ValueCache
"""



In [11]:
res=pd.read_csv('test1.csv')
for i in range(len(res)):
    row=res.iloc[i]
    print(row['ValueCache'])
# res.to_csv('test1.csv',index=False)

-
-- - -
00-00-00 - 
00-00-00 - נספח 2 - 00
00-00-00 - נספח 3 להוראה 4-01- - הוראות הפעלה ובטיחות להפעלת מכו
00-00-00 - נספח 5 לנוהל 4-01- הוראות הפעלה ובטיחות להפעלת מדפסות
1-01-01  -  נוהל רכש
1-01-01  - Appendix 7 - Elcam's Code of Ethics
1-01-01  - Purchasing
1-01-01  - נספח 1 - טופס הזמנת רכש ממוחשב וידני - עברית
1-01-01  - נספח 2  - טופס הזמנת רכש ממוחשב וידני - אנגלית
1-01-01  - נספח 3 - טופס דרישת רכש
1-01-01  - נספח 4 - טבלת חלוקת סמכויות לאישור הזמנות רכש
1-01-01  - נספח 5 - נאמני רכש מחלקתיים
1-01-01  - נספח 6 - עמידה בתקני בטיחות - רכש
1-01-01E  - Purchasing
1-01-01E - Purchasing
1-01-04  -  קליטת רכש
1-01-08  - מכירת ציוד משומש
1-01-08  - נספח 1 - מכירת ציוד משומש
1-01-09  -  החזרת פריט רכש ליצרן
1-01-10  - שילוב פעילות הרכש בפרויקטי פיתוח ותכן
1-01-11  - נספח 1 - אופי ההתקשרות עם ספק או קבמ בשלבי הפיתוח
1-01-11  - נספח 1 - אפשרויות עבודה מול קבלן משנה
1-01-11  - נספח 2 - רשימת תשומות לבחירת ספק וקב"מ
1-01-11  - תהליך קבלת החלטות לגבי make or buy
1-01-13  -  עבודה עם מחסני